In [19]:
import pandas as pd
import numpy as np
import warnings
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.metrics import mean_squared_error


def CNN_LSTM(X, y):
    reg = Sequential()
    reg.add(LSTM(units = 64,input_shape = (33,1)))
    reg.add(Dropout(0.5))
    reg.add(Dense(1))
    reg.compile(loss = 'mse', optimizer = 'adam', metrics = ['mean_squared_error'])
    reg.fit(X, y, batch_size = 8192, epochs = 5)
    test_part_x = X[0:24124]
    test_part_y = y[0:24124]
    y_pred = reg.predict(test_part_x)
    print('\nRoot Mean Squared Error: ', np.sqrt(mean_squared_error(test_part_y, y_pred)))

def main():
    item_cat_df = pd.read_csv('../Dataset_Predict_Future_Sales/item_categories.csv')
    item_df = pd.read_csv('../Dataset_Predict_Future_Sales/items.csv')
    sales_df = pd.read_csv('../Dataset_Predict_Future_Sales/sales_train.csv')
    submission_df = pd.read_csv('../Dataset_Predict_Future_Sales/sample_submission.csv')
    shops_df = pd.read_csv('../Dataset_Predict_Future_Sales/shops.csv')
    test_df = pd.read_csv('../Dataset_Predict_Future_Sales/test.csv')
    
    sales_df['date'] = pd.to_datetime(sales_df['date'], format = '%d.%m.%Y')
    sales_dt = sales_df.pivot_table(index=['shop_id', 'item_id'], values=['item_cnt_day'], columns=['date_block_num'], fill_value=0, aggfunc='sum')
    sales_dt.reset_index(inplace = True)
    sales_dt = pd.merge(test_df, sales_dt, on = ['item_id','shop_id'], how = 'left')
    sales_dt.fillna(0,inplace = True)
    sales_dt.drop(['shop_id','item_id','ID'],inplace = True, axis = 1)
    X_train = np.expand_dims(sales_dt.values[:,:-1], axis = 2)
    y_train = sales_dt.values[:,-1:]
    X_test = np.expand_dims(sales_dt.values[:,1:], axis = 2)
    CNN_LSTM(X_train, y_train)
    
if __name__ == '__main__':
    warnings.filterwarnings('ignore')
    main()
    

Epoch 1/5
214200/214200 [==============================] - 67s 311us/step - loss: 30.5379 - mean_squared_error: 30.5379
Epoch 2/5
214200/214200 [==============================] - 61s 286us/step - loss: 30.3833 - mean_squared_error: 30.3833
Epoch 3/5
214200/214200 [==============================] - 63s 296us/step - loss: 30.1970 - mean_squared_error: 30.1970
Epoch 4/5
214200/214200 [==============================] - 59s 277us/step - loss: 30.1485 - mean_squared_error: 30.1485
Epoch 5/5
214200/214200 [==============================] - 60s 281us/step - loss: 29.9881 - mean_squared_error: 29.9881

Root Mean Squared Error:  1.1335187216521547
